In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
from keras import backend as k
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16 
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, Input, Conv2D, Dropout, Flatten, BatchNormalization, Dropout, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tqdm import tqdm
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [3]:
birds = pd.read_csv('../data/birds.csv')

In [4]:
birds.head(5)

,class index,filepaths,labels,data set
0,0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train
1,0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train
2,0,train/ABBOTTS BABBLER/003.jpg,ABBOTTS BABBLER,train
3,0,train/ABBOTTS BABBLER/004.jpg,ABBOTTS BABBLER,train
4,0,train/ABBOTTS BABBLER/005.jpg,ABBOTTS BABBLER,train


In [5]:
train = birds[birds['data set'] == 'train']
test = birds[birds['data set'] == 'test']
val = birds[birds['data set'] == 'valid']

In [6]:
train.shape

(54652, 4)

In [7]:
test.shape

(1875, 4)

In [8]:
val.shape

(1875, 4)

In [9]:
BATCH_SIZE = 64

In [10]:
datagen=ImageDataGenerator(rescale=1.0/255.0)
train_generator = datagen.flow_from_dataframe(
    dataframe = train,
    directory="../data/",
    x_col="filepaths",
    y_col="labels",
    subset="training",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(256, 256),
)

valid_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=val,
    directory="../data/",
    x_col="filepaths",
    y_col="labels",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(256, 256),
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory="../data/",
    x_col="filepaths",
    y_col=None,
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(256, 256),
)

Found 54652 validated image filenames belonging to 375 classes.
Found 1875 validated image filenames belonging to 375 classes.
Found 1875 validated image filenames.


In [11]:
# ModelCheckpoint callback is used to save only the best model out of all the epochs...
checkpoint = ModelCheckpoint(filepath="../models/resnet.h5", verbose=2, save_best_only=True)

# EarlyStopping callback is used to stop the training when accuracy doesn't improve for 5 epochs...
early_stop = EarlyStopping(monitor="accuracy", min_delta=0, patience=5)

callbacks = [early_stop, checkpoint]

In [12]:
def plotter(history, model):
    fig, axes = plt.subplots(1, 2, figsize = (10, 5)) # fig of 1 row and 2 cols with 10x5 size...
    # In First column of figure, plotting accuracy and val accuracy from trained model object(history)...
    axes[0].plot(range(1, len(history.history['accuracy']) + 1), history.history['accuracy'], linestyle = 'solid', marker = 'o', color = 'crimson', label = 'Training Accuracy') 
    axes[0].plot(range(1, len(history.history['val_accuracy']) + 1), history.history['val_accuracy'], linestyle = 'solid', marker = 'o', color = 'dodgerblue', label = 'Testing Accuracy')
    axes[0].set_xlabel('Epochs', fontsize = 14)
    axes[0].set_ylabel('Accuracy',fontsize = 14)
    axes[0].set_title(f"{model} Accuracy Training vs Testing", fontsize = 14)
    axes[0].legend(loc = 'best') # Location of the legend, whereever is more empty space put legent there (loc= 'best')...
    # In Second column of figure, plotting accuracy and val accuracy from trained model object(history)...
    axes[1].plot(range(1, len(history.history['loss']) + 1), history.history['loss'], linestyle = 'solid', marker = 'o', color = 'crimson', label = 'Training Loss')
    axes[1].plot(range(1, len(history.history['val_loss']) + 1), history.history['val_loss'], linestyle = 'solid', marker = 'o', color = 'dodgerblue', label = 'Testing Loss')
    axes[1].set_xlabel('Epochs', fontsize = 14)
    axes[1].set_ylabel('Loss',fontsize = 14)
    axes[1].set_title(f"{model} Loss Training vs Testing", fontsize = 14)
    axes[1].legend(loc = 'best')
    plt.show()

In [13]:
def network(base_model):
    
    # Removes the values in the graph(network connections) but do not delete the graph itself... helps in RAM cleaning...
    tf.keras.backend.clear_session() 

    # input of size 256x256x3... hight x width = 256 x 256, number of channels = 3...
    input = Input(name = 'img_input', shape=(256, 256, 3))

    # Making Base model layers as non-trainable...
    for layer in base_model.layers:
      layer.trainable = False

    input_layer = base_model(input) # defining input layer...


    # Functional API of keras is used for this network...

    # Convolution layer with 32 filters of size 3x3 and stride = 1x1, activation function used is ReLU and "valid" padding to keep the input and output size same...
    # he normal kernel initializer is used as it performs well with non-linear activation functions like ReLU... 
    conv = Conv2D(filters = 32, kernel_size = (3, 3), strides = (1, 1), padding = 'valid', activation = 'relu', 
              kernel_initializer = tf.keras.initializers.he_normal())(input_layer)
    pool = GlobalAveragePooling2D()(conv) # the pool size is set to the input size and it outputs the average of the pool...
    d1 = Dense(units = 64, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal())(pool)
    drop1 = Dropout(0.5)(d1) # dropout was used to deactivate some of the nodes to avoid overfitting...
    d2 = Dense(units = 128, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal())(drop1)
    bn = BatchNormalization()(d2) # batch normalization was used normalize weights batch wise and in turn reduce the chances of overfitting...
    drop2 = Dropout(0.2)(bn)
    d3 = Dense(units = 64, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal())(drop2)
    Out = Dense(units = 375, activation = 'softmax', kernel_initializer = tf.keras.initializers.he_normal())(d3)
    model = tf.keras.Model(input, Out) 
    
    return model

In [ ]:
# Loading weights of the ResNet101 pre-trained model without including top layers... imagenet is a dataset on which ResNet101 was trained...
resnet_model = ResNet101(input_tensor = input, include_top=False, weights="imagenet")

res_model = network(resnet_model)

adam = Adam(learning_rate=0.001) # ADAM optimizer was used to reach the optimal weights...
res_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


# Fitting/training model...
res_history = res_model.fit(
    train_generator, 
    steps_per_epoch = len(train) / BATCH_SIZE, # number of steps in each epoch...
    callbacks=callbacks,
    epochs = 50, 
    validation_data = valid_generator)

Epoch 1/50
1708/1707 [==============================] - ETA: 0s - loss: 6.0351 - accuracy: 0.0030
Epoch 00001: val_loss improved from inf to 5.92825, saving model to ../models\resnet.h5


c:\datascience\python\machine_learning\venv\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


1707/1707 [==============================] - 287s 163ms/step - loss: 6.0351 - accuracy: 0.0030 - val_loss: 5.9283 - val_accuracy: 0.0027
Epoch 2/50
1708/1707 [==============================] - ETA: 0s - loss: 5.9234 - accuracy: 0.0042
Epoch 00002: val_loss did not improve from 5.92825
1707/1707 [==============================] - 270s 158ms/step - loss: 5.9234 - accuracy: 0.0042 - val_loss: 5.9305 - val_accuracy: 0.0027
Epoch 3/50
1708/1707 [==============================] - ETA: 0s - loss: 5.9198 - accuracy: 0.0043
Epoch 00003: val_loss did not improve from 5.92825
1707/1707 [==============================] - 270s 158ms/step - loss: 5.9198 - accuracy: 0.0043 - val_loss: 5.9328 - val_accuracy: 0.0027
Epoch 4/50
1708/1707 [==============================] - ETA: 0s - loss: 5.9183 - accuracy: 0.0045
Epoch 00004: val_loss did not improve from 5.92825
1707/1707 [==============================] - 272s 159ms/step - loss: 5.9183 - accuracy: 0.0045 - val_loss: 5.9345 - val_accuracy: 0.0027
Epoch

In [ ]:
plotter(res_history, "ResNet101")

In [ ]:
# Loading weights of the vgg16 pre-trained model without including top layers... imagenet is a dataset on which vgg16 was trained...
vgg16_model = VGG16(input_tensor = input, include_top=False, weights="imagenet")

vgg_model = network(vgg16_model)

adam = Adam(learning_rate=0.001) # ADAM optimizer was used to reach the optimal weights...
vgg_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


# Fitting/training model...
vgg_history = vgg_model.fit(
    train_generator, 
    steps_per_epoch = len(train) / BATCH_SIZE, # number of steps in each epoch...
    callbacks=callbacks,
    epochs = 50, 
    validation_data = valid_generator)

In [ ]:
plotter(vgg_history, "VGG16")

In [ ]:
# Loading weights of the InceptionV3 pre-trained model without including top layers... imagenet is a dataset on which InceptionV3 was trained...
inception_model = InceptionV3(input_tensor = input, include_top=False, weights="imagenet")

incepV3_model = network(inception_model)

adam = Adam(learning_rate=0.001) # ADAM optimizer was used to reach the optimal weights...
incepV3_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


# Fitting/training model...
incepV3_history = incepV3_model.fit(
    train_generator, 
    steps_per_epoch = len(train) / BATCH_SIZE, # number of steps in each epoch...
    callbacks=callbacks,
    epochs = 50, 
    validation_data = valid_generator)

In [ ]:
plotter(incepV3_history, "InceptionV3")